# Open Street Map - Indianapolis

Indianapolis is the capital of and largest city in Indiana. I'm from Indiana and lived in Indy for several years prior to moving to California. It has a wide geographic area with very low density, and I was curious to see how that would impact OSM data reporting.

https://www.openstreetmap.org/export#map=12/39.7683/-86.1579

## Problems in the Data

- Inconsistencies in direction abbreviations ("N" or "E" instead of "North" "East")
- Inconsistencies in road type abbreviations ("Rd" or "Hwy" instead of "Road" "Highway")

Otherwise I didn't find too many issues with the cleanliness of the data. I suspect this is likely due to a small number of users submitting the majority of data.

Fixing these issues required small modifications to the case study code. 

```python
expected = ["Street", "Avenue", "Boulevard", "Drive", "Circle", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons", "Way", "Walk", "Pike", "Run", "North", "South", "East", "West"]

mapping = { "St" : "Street",
            "Ave": "Avenue",
            "Rd" : "Road",
            "st" : "Street",
            "Us" : "US",
            "Hwy " : "Highway",
            "Dr" : "Drive",
            "Blvd" : "Boulevard",
            "W" : "West",
            "S" : "South",
            "E" : "East",
            "N" : "North"
            }
```

## Data Review

### File size
```
map.osm............190M
nodes.csv ..........72M
nodes_tags.csv.....1.3M
sample.osm..........21M
ways.csv...........5.7M
ways_nodes.csv......24M
ways_tags.csv.......14M
```

### Number of entries

- Nodes - 883,240
```SQL
SELECT COUNT(*) FROM nodes;
```
- Ways - 99,255
```SQL
SELECT COUNT(*) FROM ways;
```

### Number of unique users - 699

```SQL
SELECT COUNT(DISTINCT uid) as unique_users
FROM (
        SELECT uid
        FROM nodes
        UNION ALL
        SELECT uid
        FROM ways
    ) total;
```

### Most active users

```SQL
SELECT user, COUNT(id) as submission_count
FROM (
        SELECT id, user
        FROM nodes
        UNION ALL
        SELECT id, user
        FROM ways
    ) total
GROUP BY user
ORDER BY submission_count DESC
LIMIT 10;
```

User | Submission Count
- | -
rama_ge | 253,136
woodpeck_fixbot | 231,692
debutterfly | 92,367
"Dr Centerline" | 85,773
jbwalters22 | 60,378
svance92 | 18,085
dmsnell | 18,048
buspainter2005 | 16,576
bot-mode | 12,853
JohnnyMorbo | 11,992

## Additional Data

### Types of Places

```SQL
SELECT value, COUNT(*) as num
FROM nodes_tags
WHERE key='amenity'
GROUP BY value
HAVING COUNT(id) >= 10
ORDER BY num DESC;
```

Amenity | Count
-|-
place_of_worship|493
school|261
restaurant|177
fast_food|100
bicycle_parking|96
grave_yard|87
fuel|71
fire_station|55
fountain|51
shelter|35
parking|28
bank|27
cafe|27
library|23
post_office|23
bar|21
car_rental|21
parking_entrance|20
bench|14
pub|13
drinking_water|12
pharmacy|12
toilets|12
waste_disposal|12
atm|10
post_box|10

### Restaurants

In recent years, Indy has had a surge in the number of local restaurants. I wanted to see what kinds of restaurants are being reported and where those restaurants are.

```SQL
SELECT value, count(id) as cuisine_count
FROM nodes_tags nt
WHERE EXISTS (SELECT 1 
              FROM nodes_tags 
              WHERE value = 'restaurant'
              AND id = nt.id)
AND key = 'cuisine'
GROUP BY value
ORDER BY cuisine_count DESC
LIMIT 5;
```

Cuisine|Count
-|-
pizza|19
mexican|15
american|14
chinese|7
burger|5

Not exactly surprising, given the city. :)

```SQL
SELECT value, count(id) as zip_count
FROM nodes_tags nt
WHERE EXISTS (SELECT 1 
              FROM nodes_tags 
              WHERE value = 'restaurant'
              AND id = nt.id)
AND key = 'postcode'
GROUP BY value
ORDER BY zip_count DESC
LIMIT 5;
```

Zip Code|Count
-|-
46229|8
46203|7
46219|5
46202|3
46204|3

46203, 46202, and 46204 encompass the majority of the downtown commercial and residential areas, so it's not surprising to see them in the restaurants data. 46219 includes the Irvington neighborhood, a long-established enclave on the city's east side. 46229 is even further east and much more suburban.

I'm a little curious to see who's submitting the 46229 restaurant data.

```SQL
SELECT DISTINCT n.user
FROM nodes_tags nt
JOIN nodes n
  ON nt.id = n.id
WHERE EXISTS (SELECT 1 
              FROM nodes_tags 
              WHERE value = 'restaurant'
              AND id = nt.id)
AND key = 'postcode'
AND value = '46229';
```

jbwalters22 is the only record returned, #5 on the list of Top 10 Users above. This indicates a way that results can be skewed based on the location of the users themselves.

## Additional Analysis Opportunities

### Users and Zip Codes

I think it'd be important, especially in a geographic area dominated by small number of users, to look at the zip codes of where those users are reporting data. 